# Final Exam Image Analysis 
David Basilio Rodriguez Cortez
s231486

### Load libraries

In [1]:
from skimage import color, io
import matplotlib.pyplot as plt
import numpy as np
import pydicom as dicom
from skimage.morphology import erosion, dilation, binary_erosion, binary_dilation
from skimage.morphology import disk
from skimage.morphology import square
from skimage.filters import prewitt
from skimage.filters import median
from skimage import segmentation
from skimage import measure
import math
from scipy.stats import norm
import pandas as pd
import seaborn as sns
from skimage.transform import rescale, resize
from skimage import data, morphology, measure, segmentation, img_as_ubyte
from skimage.filters import threshold_otsu
from skimage.filters import gaussian
from skimage.color import label2rgb
from scipy.spatial import distance
from skimage.transform import rotate
from skimage.transform import EuclideanTransform
from skimage.transform import SimilarityTransform
from skimage.transform import warp
from skimage.transform import swirl
from skimage.transform import matrix_transform
import glob
from sklearn.decomposition import PCA
import random

### Load functions

In [ ]:
### AUXILIAR FUNCTION ###
def create_u_byte_image_from_vector(im_vec, height, width, channels):
    min_val = im_vec.min()
    max_val = im_vec.max()

    # Transform to [0, 1]
    im_vec = np.subtract(im_vec, min_val)
    im_vec = np.divide(im_vec, max_val - min_val)
    im_vec = im_vec.reshape(height, width, channels)
    im_out = img_as_ubyte(im_vec)
    return im_out

In [ ]:
### PCA FROM TXT FILE ###
def pca_on_single_data(x):
    n_feat = x.shape[1]
    n_obs = x.shape[0]
    print(f"Number of features: {n_feat} and number of observations: {n_obs}")

    mn = np.mean(x, axis=0)
    data = x - mn

    mins = data.min(axis=0)
    maxs = data.max(axis=0)
    spread = maxs - mins
    data = data / spread

    print(f"Answer: Amount of Sodium {data[0][1]:.2f}")         # Fixed value of data normalized
    c_x = np.cov(data.T)

    print(f"Answer: Covariance matrix at (0, 0): {c_x[0][0]:.3f}") # Fixed value of covariance matrix

    values, vectors = np.linalg.eig(c_x)

    v_norm = values / values.sum() * 100
    plt.plot(v_norm)
    plt.xlabel('Principal component')
    plt.ylabel('Percent explained variance')
    plt.ylim([0, 100])
    plt.show()

    answer = v_norm[0] + v_norm[1] + v_norm[2]              # Fixed number of PC
    print(f"Answer: Variance explained by the first three PC: {answer:.2f}")

    # Project data
    pc_proj = vectors.T.dot(data.T)

    abs_pc_proj = np.abs(pc_proj)
    max_proj_val = np.max(abs_pc_proj)                      # Fixed max value of PC
    print(f"Answer: maximum absolute projected answer {max_proj_val}")


### DETECT CHANGES IN IMAGES (BACKGROUND VS NEW FRAME) ###
def change_detection(background_path, new_frame_path):
    name_1 = background_path
    name_2 = new_frame_path

    im_1 = io.imread(name_1)
    im_2 = io.imread(name_2)
    im_1_g = color.rgb2gray(im_1)
    im_2_g = color.rgb2gray(im_2)

    average_val_org = np.average(im_1_g[150:200, 150:200]) # Fixed area
    print(f"Average_value {average_val_org:.2f}")

    alpha = 0.90
    im_new_background = alpha * im_1_g + (1 - alpha) * im_2_g
    average_val = np.average(im_new_background[150:200, 150:200]) # Fixed area
    print(f"Answer: average_value {average_val:.2f}")

    dif_thres = 0.1                                     # Fixed threshold
    dif_img = np.abs(im_new_background - im_2_g)
    dif_bin = (dif_img > dif_thres)
    io.imshow(dif_bin)
    io.show()
    changed_pixels = np.sum(dif_bin)
    print(f"Answer: Changed pixels {changed_pixels:.0f}")


### CALCULATE SYSTEM FRAME RATE ###
def system_frame_rate():
    # bytes per second
    transfer_speed = 24000000                       # Fixed transfer speed
    image_mb = 1600 * 800 * 3                       # Fixed image size
    images_per_second = transfer_speed / image_mb
    print(f"Images transfered per second {images_per_second:.3f}")

    proc_time = 0.230                               # Fixed processing time
    proc_per_second = 1/proc_time
    print(f"Images processed per second {proc_per_second:.1f}")

    max_fps = min(proc_per_second, images_per_second)
    print(f"System framerate {max_fps:.1f}")

    img_per_sec = 6.25                              # Fixed image per second
    transfer_speed = img_per_sec * image_mb
    print(f"Computed transfer speed {transfer_speed}")


### HSV THRESHOLDING ###
def nike_rgb_hsv_thresholds(img_path):
    im_name = img_path
    im_org = io.imread(im_name)
    hsv_img = color.rgb2hsv(im_org)
    hue_img = hsv_img[:, :, 0]                      # Fixed hue image

    io.imshow(hue_img)
    plt.title('Hue image')
    io.show()

    letters = (0.3 < hue_img) & (hue_img < 0.7)     # Fixed threshold
    io.imshow(letters)
    plt.title('Segmented Letters')
    io.show()

    footprint = disk(8)
    dilated = dilation(letters, footprint)

    result = dilated.sum()
    print(f"Answer: Result {result}")

    io.imshow(dilated)
    plt.title('Dilated letters')
    io.show()


### FILTERING WITH MEDIAN FILTER ###
def filtering(img_path):
    im_name = img_path
    im_org = io.imread(im_name)
    io.imshow(im_org)
    plt.title('Letters')
    io.show()

    img_g = color.rgb2gray(im_org)

    size = 8                                                        # Fixed square size
    med_img = median(img_g, square(size))
    io.imshow(med_img)
    plt.title('Filtered letters')
    io.show()
    print(f"Answer: value at (100,100): {med_img[100, 100]:.2f}")   # Fixed value at (100, 100)

    # edge_img = prewitt(img_as_ubyte(im_org))
    # min_val = edge_img.min()
    # max_val = edge_img.max()
    # io.imshow(edge_img, vmin=min_val, vmax=max_val, cmap="terrain")
    # plt.title('Prewitt filtered image')
    # io.show()
    #
    # bin_edges = edge_img > 0.06
    # io.imshow(bin_edges)
    # plt.title('Binary edges. Manual threshold')
    # io.show()
    #
    # num_pixels = bin_edges.sum()
    # print(f"Number of edge pixels {num_pixels}")


### BLOB CLASSIFICATION ###
def letters_blob_analysis(img_path):
    im_name = img_path
    im_org = io.imread(im_name)
    io.imshow(im_org)
    plt.title('Letters')
    io.show()

    img_r = im_org[:, :, 0]
    img_g = im_org[:, :, 1]
    img_b = im_org[:, :, 2]

    img_letters = (img_r > 100) & (img_g < 100) & (img_b < 100)     # Fixed threshold

    footprint = disk(3)                                             # Fixed disk size
    eroded = erosion(img_letters, footprint)                        # Fixed morphology operation

    result = eroded.sum()
    print(f"Answer: eroded {result}")

    # img_g = color.rgb2gray(im_org)

    # size = 8
    # footprint = np.ones([size, size])
    # med_img = median(img_g, footprint)
    io.imshow(eroded)
    plt.title('Filtered letters')
    io.show()
    # print(f"Answer: value at (100,100): {med_img[100, 100]:.2f}")

    label_img = measure.label(eroded)
    n_labels = label_img.max()
    print(f"Number of labels: {n_labels}")

    region_props = measure.regionprops(label_img)

    # areas = np.array([prop.area for prop in region_props])

    min_area = 1000                                                # Fixed areas and perimeter                                             
    max_area = 4000
    min_perm = 300

    # Create a copy of the label_img
    label_img_filter = label_img.copy()
    for region in region_props:
        a = region.area
        p = region.perimeter

        if p < min_perm or a < min_area or a > max_area:
            # set the pixels in the invalid areas to background
            for cords in region.coords:
                label_img_filter[cords[0], cords[1]] = 0

    # Create binary image from the filtered label image
    i_letter = label_img_filter > 0
    # show_comparison(img, i_area, 'Found spleen based on area')
    io.imshow(i_letter)
    io.show()


### DICOM AND ROI ANALYSIS ###
def kidney_pixel_analysis(data_path):
    in_dir = data_path
    im_name = "1-166.dcm"                                       # Fixed dicom name             

    ct = dicom.read_file(in_dir + im_name)
    img = ct.pixel_array

    kidney_l_roi = io.imread(in_dir + 'kidneyROI_l.png')        # Fixed ROI name
    kidney_l_mask = kidney_l_roi > 0
    kidney_l_values = img[kidney_l_mask]
    (mu_kidney_l, std_kidney_l) = norm.fit(kidney_l_values)
    print(f"Answer: kidney_l: Average {mu_kidney_l:.0f} standard deviation {std_kidney_l:.0f}")

    kidney_r_roi = io.imread(in_dir + 'kidneyROI_r.png')        # Fixed ROI name
    kidney_r_mask = kidney_r_roi > 0
    kidney_r_values = img[kidney_r_mask]
    (mu_kidney_r, std_kidney_r) = norm.fit(kidney_r_values)
    print(f"Answer: kidney_r: Average {mu_kidney_r:.0f} standard deviation {std_kidney_r:.0f}")


    #
    # aorta_roi = io.imread(in_dir + 'AortaROI.png')
    # aorta_mask = aorta_roi > 0
    # aorta_values = img[aorta_mask]
    # (mu_aorta, std_aorta) = norm.fit(aorta_values)
    # print(f"Aorta: Average {mu_aorta:.0f} standard deviation {std_aorta:.0f}")
    #
    liver_roi = io.imread(in_dir + 'LiverROI.png')              # Fixed ROI name
    liver_mask = liver_roi > 0
    liver_values = img[liver_mask]
    (mu_liver, std_liver) = norm.fit(liver_values)
    # print(f"Answer: liver: Average {mu_liver:.0f} standard deviation {std_liver:.0f}")
    # min_hu = mu_liver - np.sqrt(3) * std_liver
    # max_hu = mu_liver + np.sqrt(3) * std_liver
    min_hu = mu_liver - std_liver
    max_hu = mu_liver + std_liver
    print(f"Answer: HU limits : {min_hu:0.2f} {max_hu:0.2f}")

    bin_img = (img > min_hu) & (img < max_hu)
    liver_label_colour = color.label2rgb(bin_img)
    io.imshow(liver_label_colour)
    plt.title("First Liver estimate")
    io.show()

    footprint = disk(3)                                         # Fixed disk size                                
    dilated = dilation(bin_img, footprint)

    footprint = disk(10)                                        # Fixed disk size            
    eroded = erosion(dilated, footprint)
    io.imshow(eroded)
    plt.title("Second Liver estimate")
    io.show()

    footprint = disk(10)                                        # Fixed disk size
    dilated = dilation(eroded, footprint)
    io.imshow(dilated)
    plt.title("Third Liver estimate")
    io.show()

    label_img = measure.label(dilated)
    n_labels = label_img.max()
    print(f"Number of labels: {n_labels}")

    region_props = measure.regionprops(label_img)

    min_area = 1500                                            # Fixed areas and perimeter      
    max_area = 7000
    min_perm = 300

    # Create a copy of the label_img
    label_img_filter = label_img.copy()
    for region in region_props:
        a = region.area
        p = region.perimeter

        if p < min_perm or a < min_area or a > max_area:
            # set the pixels in the invalid areas to background
            for cords in region.coords:
                label_img_filter[cords[0], cords[1]] = 0

    # Create binary image from the filtered label image
    i_liver = label_img_filter > 0
    # show_comparison(img, i_area, 'Found spleen based on area')
    io.imshow(i_liver)
    io.show()

    gt_bin = liver_roi > 0
    dice_score = 1 - distance.dice(i_liver.ravel(), gt_bin.ravel())
    print(f"Answer: DICE score {dice_score:.3f}")


    #
    # min_hu = 147
    # max_hu = 155
    # hu_range = np.arange(min_hu, max_hu, 1.0)
    # pdf_aorta = norm.pdf(hu_range, mu_aorta, std_aorta)
    # pdf_liver = norm.pdf(hu_range, mu_liver, std_liver)
    # plt.plot(hu_range, pdf_aorta, 'r--', label="aorta")
    # plt.plot(hu_range, pdf_liver, 'g', label="liver")
    # plt.title("Fitted Gaussians")
    # plt.legend()
    # plt.show()
    # # Answer = 151


### OTSU THRESHOLDING ###
def otsu_rotate_image(img_path):
    im_name = img_path
    im_org = io.imread(im_name)

    # angle in degrees - counter clockwise
    rotation_angle = 11                                     # Fixed rotation angle           
    rot_center = [40, 40]                                   # Fixed rotation center    
    rotated_img = rotate(im_org, rotation_angle, center=rot_center)
    # rot_byte = img_as_ubyte(rotated_img
    # print(f"Value at (200, 200) : {rot_byte[200, 200]}")
    # io.imshow(rot_byte)
    # io.show()

    img_gray = color.rgb2gray(rotated_img)

    auto_tresh = threshold_otsu(img_gray)
    print(f"Answer: Otsus treshold: {auto_tresh:.2f}")

    # auto_tresh = threshold_otsu(img_out)
    # print(f"Otsus treshold {auto_tresh:.2f}")

    img_thres = img_gray > auto_tresh
    io.imshow(img_thres)
    io.show()
    for_percent = img_thres.sum() / img_thres.size * 100

    print(f"Answer: foreground pixels percent: {for_percent:.0f}")


### LANDMARK BASED REGISTRATION ###
def landmark_based_registration(src_path, dst_path):
    src_img = io.imread(src_path)
    dst_img = io.imread(dst_path)

    # src = np.array([[55, 220], [675, 105], [675, 315]])
    # dst = np.array([[165, 100], [605, 200], [525, 379]])

    # src = np.array([[320, 40], [120, 425], [330, 740]])
    # dst = np.array([[320, 80], [155, 380], [300, 670]])

    src = np.array([[40, 320], [425, 120], [740, 330]])             # Fixed landmarks
    dst = np.array([[80, 320], [380, 155], [670, 300]])

    e_x = src[:, 0] - dst[:, 0]
    error_x = np.dot(e_x, e_x)
    e_y = src[:, 1] - dst[:, 1]
    error_y = np.dot(e_y, e_y)
    f = error_x + error_y
    print(f"Landmark alignment error F: {f}")

    plt.imshow(src_img)
    plt.plot(src[:, 0], src[:, 1], '.r', markersize=12)
    plt.title("Source image")
    plt.show()

    fig, ax = plt.subplots()
    ax.plot(src[:, 0], src[:, 1], '*r', markersize=12, label="Source")
    ax.plot(dst[:, 0], dst[:, 1], '*g', markersize=12, label="Destination")
    ax.invert_yaxis()
    ax.legend()
    ax.set_title("Landmarks before alignment")
    plt.show()

    # plt.scatter(src[:, 0], src[:, 1])
    # plt.scatter(trg[:, 0], trg[:, 1])
    # plt.show()
    # tform = EuclideanTransform()
    tform = SimilarityTransform()                                # Fixed transform              
    tform.estimate(src, dst)
    print(f"Answer: scale {tform.scale:.2f}")

    src_transform = matrix_transform(src, tform.params)
    # print(src_transform)

    e_x = src_transform[:, 0] - dst[:, 0]
    error_x = np.dot(e_x, e_x)
    e_y = src_transform[:, 1] - dst[:, 1]
    error_y = np.dot(e_y, e_y)
    f_after = error_x + error_y
    print(f"Aligned landmark alignment error F: {f_after}")
    print(f"Answer: alignment error change: {f - f_after:.0f}")

    fig, ax = plt.subplots()
    ax.plot(src[:, 0], src[:, 1], '*r', markersize=12, label="Source")
    ax.plot(src_transform[:, 0], src_transform[:, 1], '*b', markersize=12, label="Source transformed")
    ax.plot(dst[:, 0], dst[:, 1], '*g', markersize=12, label="Destination")
    ax.invert_yaxis()
    ax.legend()
    ax.set_title("Landmarks after alignment")
    plt.show()

    warped = warp(src_img, tform.inverse)

    val_1 = img_as_ubyte(warped)[200, 200]                      # Fixed pixel coordinates
    val_2 = img_as_ubyte(dst_img)[200, 200]
    print(f"Value at (200, 200) : {val_1}")
    print(f"Value at (200, 200) : {val_2}")

    val_1_b = val_1[2]
    val_2_b = val_2[2]
    print(f"Answer: b difference {np.abs(val_2_b - val_1_b)}")

    fig, ax = plt.subplots(ncols=3, figsize=(16, 6))
    ax[0].imshow(src_img)
    ax[0].plot(src[:, 0], src[:, 1], '.r', markersize=12)
    # ax[1].plot(dst[:, 0], dst[:, 1], '.r', markersize=12)
    ax[1].imshow(warped)
    ax[1].plot(src_transform[:, 0], src_transform[:, 1], '.r', markersize=12)
    ax[2].imshow(dst_img)
    ax[2].plot(dst[:, 0], dst[:, 1], '.r', markersize=12)
    for a in ax:
        a.axis('off')
    plt.tight_layout()
    plt.show()




In [ ]:
### PCA ALL IMAGES ### 
def do_pca_on_all_images_in_directory(data_path):
    # Find all image files in data dir
    in_dir = data_path
    all_images = glob.glob(in_dir + "*.png")            # Fixed image extension
    n_samples = len(all_images)

    # Exercise 2
    # Read first image to get image dimensions
    im_org = io.imread(all_images[0])
    im_shape = im_org.shape
    height = im_shape[0]
    width = im_shape[1]
    channels = im_shape[2]
    n_features = height * width * channels

    print(f"Found {n_samples} image files. Height {height} Width {width} Channels {channels} n_features {n_features}")

    data_matrix = np.zeros((n_samples, n_features))

    idx = 0
    for image_file in all_images:
        img = io.imread(image_file)
        flat_img = img.flatten()
        data_matrix[idx, :] = flat_img
        idx += 1

    # Exercise 3 + 4: The Average image
    average_pizza = np.mean(data_matrix, 0)
    io.imshow(create_u_byte_image_from_vector(average_pizza, height, width, channels))
    plt.title('The Average Pizza')
    io.show()

    # Find the missing pizza twin
    # Exercise 7 + 8
    # im_miss = io.imread("data/pizzaPCA/super_pizza.png")
    # im_miss_flat = im_miss.flatten()

    # Find pizza closest to the average pizza
    sub_data = data_matrix - average_pizza
    sub_distances = np.linalg.norm(sub_data, axis=1)

    # Exercise 9 + 10 + 11
    best_match = np.argmin(sub_distances)
    best_average_pizza = data_matrix[best_match, :]
    worst_match = np.argmax(sub_distances)
    worst_average_pizza = data_matrix[worst_match, :]

    print(f"Pizza most away from average pizza {worst_match} : {all_images[worst_match]}")

    fig, ax = plt.subplots(ncols=3, figsize=(16, 6))
    ax[0].imshow(create_u_byte_image_from_vector(average_pizza, height, width, channels))
    ax[0].set_title('The Real average pizza')
    ax[1].imshow(create_u_byte_image_from_vector(best_average_pizza, height, width, channels))
    ax[1].set_title('The Best Matching Twin pizza')
    ax[2].imshow(create_u_byte_image_from_vector(worst_average_pizza, height, width, channels))
    ax[2].set_title('The Worst Matching Twin pizza')
    for a in ax:
        a.axis('off')
    plt.tight_layout()
    plt.show()

    # Exercise 12
    print("Computing PCA")
    # pizzas_pca = PCA(n_components=10)
    pizzas_pca = PCA(n_components=5)
    # pizzas_pca = PCA(n_components=0.80)
    pizzas_pca.fit(data_matrix)

    # Exercise 13
    plt.plot(pizzas_pca.explained_variance_ratio_ * 100)
    plt.xlabel('Principal component')
    plt.ylabel('Percent explained variance')
    plt.show()

    # Exercise 14
    print(f"Total variation explained by first component {pizzas_pca.explained_variance_ratio_[0] * 100}")
    print(f"Number of component to explain desired variation {pizzas_pca.n_components_}")

    # Exercise 15
    components = pizzas_pca.transform(data_matrix)

    # Exercise 16
    pc_1 = components[:, 0]
    pc_2 = components[:, 1]

    # Debug
    print(f"PC_1 : {pc_1.min()} - {pc_1.max()}")
    print(f"PC_2 : {pc_2.min()} - {pc_2.max()}")
    print(f"Explained variance: {pizzas_pca.explained_variance_[0]} {pizzas_pca.explained_variance_[1]}")
    print(f"SQRT Explained variance: {np.sqrt(pizzas_pca.explained_variance_[0])} "
          f"{np.sqrt(pizzas_pca.explained_variance_[1])}")

    plt.plot(pc_1, pc_2, '.')
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.show()

    # Exercise 17 + 18
    extreme_pc_1_pizza_m = np.argmin(pc_1)
    extreme_pc_1_pizza_p = np.argmax(pc_1)
    extreme_pc_2_pizza_m = np.argmin(pc_2)
    extreme_pc_2_pizza_p = np.argmax(pc_2)

    print(f'PC 1 extreme minus pizza: {all_images[extreme_pc_1_pizza_m]}')
    print(f'PC 1 extreme minus pizza: {all_images[extreme_pc_1_pizza_p]}')
    print(f'PC 2 extreme minus pizza: {all_images[extreme_pc_2_pizza_m]}')
    print(f'PC 2 extreme minus pizza: {all_images[extreme_pc_2_pizza_p]}')

    fig, ax = plt.subplots(ncols=4, figsize=(16, 6))
    ax[0].imshow(create_u_byte_image_from_vector(data_matrix[extreme_pc_1_pizza_m, :], height, width, channels))
    ax[0].set_title(f'PC 1 extreme minus pizza')
    ax[1].imshow(create_u_byte_image_from_vector(data_matrix[extreme_pc_1_pizza_p, :], height, width, channels))
    ax[1].set_title(f'PC 1 extreme plus pizza')
    ax[2].imshow(create_u_byte_image_from_vector(data_matrix[extreme_pc_2_pizza_m, :], height, width, channels))
    ax[2].set_title(f'PC 2 extreme minus pizza')
    ax[3].imshow(create_u_byte_image_from_vector(data_matrix[extreme_pc_2_pizza_p, :], height, width, channels))
    ax[3].set_title(f'PC 2 extreme plus pizza')
    for a in ax:
        a.axis('off')
    plt.tight_layout()
    plt.show()

    plt.plot(pc_1, pc_2, '.', label="All pizzas")
    plt.plot(pc_1[extreme_pc_1_pizza_m], pc_2[extreme_pc_1_pizza_m], "*", color="green", label="Extreme pizza 1 minus")
    plt.plot(pc_1[extreme_pc_1_pizza_p], pc_2[extreme_pc_1_pizza_p], "*", color="green", label="Extreme pizza 1 plus")
    plt.plot(pc_1[extreme_pc_2_pizza_m], pc_2[extreme_pc_2_pizza_m], "*", color="green", label="Extreme pizza 2 minus")
    plt.plot(pc_1[extreme_pc_2_pizza_p], pc_2[extreme_pc_2_pizza_p], "*", color="green", label="Extreme pizza 2 plus")
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.title("pizzas in PCA space")
    plt.legend()
    plt.show()

    # Exercise 19 + 20: First synthetic pizza
    w = 60000                                                                           # Fixed weight             
    synth_pizza = average_pizza + w * pizzas_pca.components_[0, :]
    io.imshow(create_u_byte_image_from_vector(synth_pizza, height, width, channels))
    plt.title("First synthetic pizza")
    plt.show()

    # Exercise 22: Major modes of variation
    n_modes = 5
    # Create a n_modes x 3 plot to show major modes of variation
    fig, ax = plt.subplots(ncols=3, nrows=n_modes, figsize=(15, 15))
    for m in range(n_modes):
        # Average pizza in middle of all
        ax[m][1].set_title("The Mean pizza")
        ax[m][1].imshow(create_u_byte_image_from_vector(average_pizza, height, width, channels))
        # Create mode: synth_pizza = average_pizza + alpha * eigenvector[m]
        synth_pizza_plus = average_pizza + 3 * np.sqrt(pizzas_pca.explained_variance_[m]) * pizzas_pca.components_[m, :]
        synth_pizza_minus = average_pizza - 3 * np.sqrt(pizzas_pca.explained_variance_[m]) * pizzas_pca.components_[m, :]
        ax[m][0].set_title(f"Mode: {m} minus")
        ax[m][2].set_title(f"Mode: {m} plus")
        ax[m][0].imshow(create_u_byte_image_from_vector(synth_pizza_minus, height, width, channels))
        ax[m][2].imshow(create_u_byte_image_from_vector(synth_pizza_plus, height, width, channels))
        ax[m][0].axis('off')
        ax[m][1].axis('off')
        ax[m][2].axis('off')
    fig.suptitle("Major modes of pizza variations")
    plt.tight_layout()
    plt.show()

    print(f"Computing synthetic pizzas")
    n_random = 3
    n_components_to_use = 10
    n_components_to_use = min(n_components_to_use, pizzas_pca.n_components_)
    fig, ax = plt.subplots(ncols=n_random, nrows=n_random, figsize=(10, 10))
    for i in range(n_random):
        for j in range(n_random):
            synth_pizza = average_pizza
            for idx in range(n_components_to_use):
                w = random.uniform(-1, 1) * 3 * np.sqrt(pizzas_pca.explained_variance_[idx])
                synth_pizza = synth_pizza + w * pizzas_pca.components_[idx, :]
            ax[i][j].imshow(create_u_byte_image_from_vector(synth_pizza, height, width, channels))
            ax[i][j].axis('off')
    fig.suptitle("Some Random Synthetic pizzas")
    plt.tight_layout()
    plt.show()

    # Exercise 24: Find the missing pizza twin
    im_miss = io.imread("data/pizzaPCA/super_pizza.png")                                # Fixed missing pizza
    im_miss_flat = im_miss.flatten()
    im_miss_flat = im_miss_flat.reshape(1, -1)
    pca_coords = pizzas_pca.transform(im_miss_flat)
    pca_coords = pca_coords.flatten()

    # Exercise 25
    plt.plot(pc_1, pc_2, '.', label="All pizzas")
    plt.plot(pca_coords[0], pca_coords[1], "*", color="red", label="Missing pizza")
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.title("The Missing pizza in PCA space")
    plt.legend()
    plt.show()

    # Exercise 26
    synth_pizza = average_pizza
    for idx in range(pizzas_pca.n_components_):
        synth_pizza = synth_pizza + pca_coords[idx] * pizzas_pca.components_[idx, :]

    fig, ax = plt.subplots(ncols=2, figsize=(16, 6))
    ax[0].imshow(im_miss)
    ax[0].set_title('The Real Missing pizza')
    ax[1].imshow(create_u_byte_image_from_vector(synth_pizza, height, width, channels))
    ax[1].set_title('The Synthetic Missing pizza')
    for a in ax:
        a.axis('off')
    plt.tight_layout()
    plt.show()

    # Exercise 27
    comp_sub = components - pca_coords
    pca_distances = np.linalg.norm(comp_sub, axis=1)

    best_match = np.argmin(pca_distances)
    print(f"Answer: Best matching PCA pizza {all_images[best_match]}")
    best_twin_pizza = data_matrix[best_match, :]
    worst_match = np.argmax(pca_distances)
    worst_twin_pizza = data_matrix[worst_match, :]
    fig, ax = plt.subplots(ncols=3, figsize=(16, 6))
    ax[0].imshow(im_miss)
    ax[0].set_title('The Real Missing pizza')
    ax[1].imshow(create_u_byte_image_from_vector(best_twin_pizza, height, width, channels))
    ax[1].set_title('The Best Matching Twin pizza')
    ax[2].imshow(create_u_byte_image_from_vector(worst_twin_pizza, height, width, channels))
    ax[2].set_title('The Worst Matching Twin pizza')
    for a in ax:
        a.axis('off')
    plt.tight_layout()
    plt.show()

    # Exercise 28
    n_best = 5
    best = np.argpartition(pca_distances, n_best)
    fig, ax = plt.subplots(ncols=n_best, figsize=(16, 6))
    for i in range(n_best):
        candidate_twin_pizza = data_matrix[best[i], :]
        ax[i].imshow(create_u_byte_image_from_vector(candidate_twin_pizza, height, width, channels))
        ax[i].axis('off')

    fig.suptitle(f"The {n_best} most similar pizzas")
    plt.tight_layout()
    plt.show()


### Question 1

### Question 2

### Question 3

### Question 4

### Question 5

### Question 6

### Question 7

### Question 8

### Question 9

### Question 10

### Question 11

### Question 12

### Question 13

### Question 14

### Question 15

### Question 16

### Question 17

### Question 18

### Question 19

### Question 20

### Question 21

### Question 22

### Question 23

### Question 24

### Question 25